# Advent of Code 2018 - Day 8

This one took some shots, but I am ultimately happy with the solution. It is naturally recursive.

In [1]:
data = []
with open('inputs_day_8.txt', 'r') as f:
  for line in f:
    data.append(line.strip())

len(data[0])

33500

In [2]:
numbers = [int(x) for x in data[0].split(' ')]
numbers[ : 20]

[8, 11, 7, 2, 4, 5, 3, 6, 1, 6, 0, 8, 1, 7, 6, 3, 1, 3, 1, 1]

## Part 1

In [3]:
# Header is a tuple of two numbers
class Node():
  def __init__(self, header):
    self.header = header
    self.children = []
    self.metadata_entries = []


In [4]:
# Given a tree, figure out how long the code for it is
# It is (kind of) the inverse of the problem, but it helps with building a tree
# It removes the need for keeping track of length in the nodes

def tree_code_length(node):
  length = 2 + node.header[1]

  for child in node.children:
    length += tree_code_length(child)

  return length

In [5]:
# Recursively build a tree given a tape (list of numbers) and an index indicating the location of the header for a node to make
def build_tree(numbers, index):
  #print(*numbers[ : index], numbers[index : index + 2], *numbers[index + 2 :])

  num_children = numbers[index]
  num_meta = numbers[index + 1]
  #print(num_children, num_meta)
  
  if num_children == 0: # Leaf node
    leaf = Node((num_children, num_meta))
    leaf.metadata_entries = numbers[index + 2 : index + 2 + num_meta]
    return leaf

  else:
    parent = Node((num_children, num_meta))

    # Next, we recursivley build trees for the children
    # This needs to be done before we can figure out where on the tape to read the metadata from
    child_index = index + 2 # First child defined right after header
    for i in range(num_children):
      child_tree = build_tree(numbers, child_index)
      parent.children.append(child_tree)
      # The next child is found by counting how long the current subtree took
      child_index += tree_code_length(child_tree)

    # Add node metadata
    parent.metadata_entries = numbers[child_index : child_index + num_meta]

    return parent

root = build_tree(numbers, 0)

In [14]:
from graphviz import Digraph

g = Digraph('G', filename='viz')

def graph(node, g):
  g.node(str(node), label = str(node.header))

  for child in node.children:
    g.edge(str(node), str(child))
    graph(child, g)


graph(root, g)
g.view()

'viz.pdf'

In [6]:
def get_total_metadata_entries(node):

  total = [x for x in node.metadata_entries] # Copies it to avoid modifying it, Ughh Python!
  
  for child in node.children:
    a = get_total_metadata_entries(child)
    total += a

  return total

sum(get_total_metadata_entries(root))

42472

## Part 2

In [7]:
def value(node):

  if len(node.children) == 0:
    return sum(node.metadata_entries)

  else:
    res = 0
    for i in node.metadata_entries:
      if i - 1 in range(len(node.children)): # Only entries that are indexes (starting at 1, not 0) of children
         res += value(node.children[i - 1])
    return res

value(root)

21810